In [ ]:
import cv2
import csv
import pandas as pd
from pandas import DataFrame
from sklearn import preprocessing
import os

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.python.keras.models import Sequentiasl
from tensorflow.python.keras.layers import InputLayer, Input
from tensorflow.python.keras.layers import Reshape, MaxPooling2D
from tensorflow.python.keras.layers import Conv2D, Dense, Flatten
from tensorflow.python.keras.layers import Dropout

In [ ]:
# Training Labels
train_data= pd.read_csv("/Dataset/meta-data/train.csv")

x1 = train_data.values
y_train = x1[:13000,1:]
y_train = pd.DataFrame(y_train)

le = preprocessing.LabelEncoder()
y_train = y_train.apply(le.fit_transform)
y_train = np.array(y_train)

In [ ]:
print(y_train[0])
print(y_train.shape)

In [ ]:
filelist_train = []
image_train = []
filelist_test = []
image_test = []
for i in range(1,13001):
    filelist_train.append("/Dataset/train//Img-"+str(i)+".jpg")
    image_train.append("Img-"+str(i)+".jpg")
for i in range(1,6001):
    filelist_test.append("/Dataset/test//Img-"+str(i)+".jpg")
    image_test.append("Img-"+str(i)+".jpg")

In [ ]:
"""
print(filelist_train[0])
print(image_train[0])
print(filelist_test[0])
print(image_test[0])
"""

In [ ]:
Img_size = 224

In [ ]:
x_train = np.array([np.array(cv2.resize(cv2.imread(fname),(Img_size,Img_size))) for fname in filelist_train])

In [ ]:
x_test = np.array([np.array(cv2.resize(cv2.imread(fname),(Img_size,Img_size))) for fname in filelist_test])

In [ ]:
print(x_train.shape)
print(x_test.shape)

In [ ]:
import inception

In [ ]:
model = inception.Inception()

In [ ]:
from inception import transfer_values_cache

In [ ]:
file_path_cache_train_image = os.path.join('/inception_train_jpg.pkl')
file_path_cache_test_image = os.path.join('/inception_test_jpg.pkl')

In [ ]:
print("Processing Inception train transfer-values")

images_scaled = x_train * 255.0

# If transfer-values have already been calculated then reload them,
# otherwise calculate them and save them to a cache-file.
transfer_values_train_jpg = transfer_values_cache(cache_path=file_path_cache_train_image,
                                             images=images_scaled,
                                             model=model)

In [ ]:
print("Processing Inception test transfer-values")

images_scaled = x_test * 255.0

# If transfer-values have already been calculated then reload them,
# otherwise calculate them and save them to a cache-file.
transfer_values_test_jpg = transfer_values_cache(cache_path=file_path_cache_test_image,
                                             images=images_scaled,
                                             model=model)

In [ ]:
labels = ['antelope','bat','beaver','bobcat','buffalo','chihuahua','chimpanzee','collie','dalmatian',
          'german+shepherd','grizzly+bear','hippopotamus','horse','killer+whale','mole','moose'
          ,'mouse','otter','ox','persian+cat','raccoon','rat','rhinoceros','seal','siamese+cat'
          ,'spider+monkey','squirrel','walrus','weasel','wolf']

In [ ]:
model = Sequential()
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(30, activation='softmax'))

In [ ]:
from tensorflow.python.keras.optimizers import Adam

optimizer = Adam(lr = 1e-4)

In [ ]:
model.compile(optimizer=optimizer,
             loss = 'categorical_crossentropy',
             metrics=['accuracy'])

In [ ]:
cls = np.eye(30)[y_train]

In [ ]:
cls = cls.reshape(13000,30)

In [ ]:
model.fit(x=transfer_values_train_jpg,
         y=cls,
         epochs=550, batch_size=28)

In [ ]:
result = model.evaluate(x= transfer_values_train_jpg,
                       y = cls)

In [ ]:
for name,value in zip(model.metrics_names, result):
    print(name,value)

In [ ]:
print(y_train[0])
print(labels[int(y_train[0])])

In [ ]:
y_pred = model.predict(x=transfer_values_train_jpg)

In [ ]:
cls_pred = np.argmax(y_pred, axis=1)
print(labels[int(cls_pred[0])])

In [ ]:
print(y_pred[1])

In [ ]:
y_pred_test = model.predict(x=transfer_values_test_jpg)

In [ ]:
cls_pred_test = np.argmax(y_pred_test, axis=1)
print(cls_pred_test[0])
print(labels[int(cls_pred_test[0])])

In [ ]:
print(y_pred_test[0])

In [ ]:
print(y_pred_test[0][4])

In [ ]:
df = pd.DataFrame(columns =['image_id','antelope','bat','beaver','bobcat','buffalo','chihuahua','chimpanzee','collie','dalmatian','german+shepherd','grizzly+bear','hippopotamus'
,'horse'
,'killer+whale'
,'mole'
,'moose'
,'mouse'
,'otter'
,'ox'
,'persian+cat'
,'raccoon'
,'rat'
,'rhinoceros'
,'seal'
,'siamese+cat'
,'spider+monkey'
,'squirrel'
,'walrus'
,'weasel'
,'wolf'])

In [ ]:
df.to_csv('/home/ubuntu/submission_file.csv')

In [ ]:
for k in range(6000):
    ls = []
    ls.append(image_test[k])
    for i in range(30):
        ls.append(y_pred_test[k][i])
    with open('/home/ubuntu/submission_file.csv','a') as f:
        writer = csv.writer(f)
        writer.writerow(ls)